# Feature Engineering

In [1]:
import pandas as pd

# disable chained assignments
pd.options.mode.chained_assignment = None 

## Clean Dataset

In [2]:
# import dataset
cleaning = pd.read_csv('../Data/Bitcoin_sentiment.csv')
cleaning.head()

,Unnamed: 0,time,avg_positive,avg_neutral,avg_negative,avg_compound
0,0,2018-12-31,0.138213,0.791725,0.070078,0.133069
1,1,2019-01-01,0.133899,0.801962,0.064177,0.131864
2,2,2019-01-02,0.124542,0.805365,0.070119,0.111221
3,3,2019-01-03,0.145012,0.785654,0.069367,0.141588
4,4,2019-01-04,0.124144,0.801732,0.074139,0.123028


In [3]:
# Drop unnecessary columns and set time column to datetime
cleaning.drop(cleaning.columns[[0]], axis=1, inplace=True)
cleaning['time'] = pd.to_datetime(cleaning['time'])
cleaning.dtypes

time            datetime64[ns]
avg_positive           float64
avg_neutral            float64
avg_negative           float64
avg_compound           float64
dtype: object

In [4]:
# Set index as time column
cleaning.set_index('time', inplace=True)
cleaning.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 957 entries, 2018-12-31 to 2021-09-08
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   avg_positive  957 non-null    float64
 1   avg_neutral   957 non-null    float64
 2   avg_negative  957 non-null    float64
 3   avg_compound  957 non-null    float64
dtypes: float64(4)
memory usage: 37.4 KB


In [5]:
# replace all missing datetime values in index and fill with missing date and mean of values per column
cleaning.resample('D').mean()
cleaning.reset_index(inplace=True)
cleaning.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound
0,2018-12-31,0.138213,0.791725,0.070078,0.133069
1,2019-01-01,0.133899,0.801962,0.064177,0.131864
2,2019-01-02,0.124542,0.805365,0.070119,0.111221
3,2019-01-03,0.145012,0.785654,0.069367,0.141588
4,2019-01-04,0.124144,0.801732,0.074139,0.123028


## Bitcoin Ticker Symbol Price

In [6]:
## Plot ticker symbol price
btc_df = pd.read_csv('../Data/Historical/BTC.csv')
btc_df.head()
btc_df.dtypes

time       object
low       float64
high      float64
open      float64
close     float64
volume    float64
dtype: object

In [7]:
# Make time column a series with dtype datetime64[ns]
btc_df['time'] = pd.to_datetime(btc_df['time'])

In [8]:
#greater than the start date and smaller than the end date
mask = (btc_df['time'] >= '2018-12-31T00:00') & (btc_df['time'] <= '2021-9-8T00:00')
mask_df = btc_df.loc[mask]
mask_df.head()

,time,low,high,open,close,volume
1260,2018-12-31,3625.00,3835.40,3829.00,3691.86,12799.942626
1261,2019-01-01,3651.02,3841.17,3691.87,3826.10,10812.884988
1262,2019-01-02,3770.07,3916.57,3826.10,3890.79,9982.470846
1263,2019-01-03,3758.07,3893.80,3890.80,3787.57,9327.647089
1264,2019-01-04,3730.00,3849.00,3787.57,3820.82,9225.150500


## Merge Dataframes

In [9]:
# Convert time column to datetime
cleaning['time'] = pd.to_datetime(cleaning['time'])
df_merged = pd.merge(cleaning, mask_df, how='outer', on='time')
df_merged.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound,low,high,open,close,volume
0,2018-12-31,0.138213,0.791725,0.070078,0.133069,3625.00,3835.40,3829.00,3691.86,12799.942626
1,2019-01-01,0.133899,0.801962,0.064177,0.131864,3651.02,3841.17,3691.87,3826.10,10812.884988
2,2019-01-02,0.124542,0.805365,0.070119,0.111221,3770.07,3916.57,3826.10,3890.79,9982.470846
3,2019-01-03,0.145012,0.785654,0.069367,0.141588,3758.07,3893.80,3890.80,3787.57,9327.647089
4,2019-01-04,0.124144,0.801732,0.074139,0.123028,3730.00,3849.00,3787.57,3820.82,9225.150500


In [10]:
df_merged.dtypes

time            datetime64[ns]
avg_positive           float64
avg_neutral            float64
avg_negative           float64
avg_compound           float64
low                    float64
high                   float64
open                   float64
close                  float64
volume                 float64
dtype: object

In [11]:
df_merged.to_csv('../Data/merged_sentiment.csv')

In [12]:
test = df_merged
test.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound,low,high,open,close,volume
0,2018-12-31,0.138213,0.791725,0.070078,0.133069,3625.00,3835.40,3829.00,3691.86,12799.942626
1,2019-01-01,0.133899,0.801962,0.064177,0.131864,3651.02,3841.17,3691.87,3826.10,10812.884988
2,2019-01-02,0.124542,0.805365,0.070119,0.111221,3770.07,3916.57,3826.10,3890.79,9982.470846
3,2019-01-03,0.145012,0.785654,0.069367,0.141588,3758.07,3893.80,3890.80,3787.57,9327.647089
4,2019-01-04,0.124144,0.801732,0.074139,0.123028,3730.00,3849.00,3787.57,3820.82,9225.150500


In [13]:
# Create blank column in test dataframe
test['price_difference'] = ''

In [14]:
# Insert 1 or 0 depending on if the closing price went up or down
num = 1
T = int(1)
R = int(0)


try:

    for index in test.iterrows():

        # Calculate difference between two columns
        n = test['close'].iloc[T] - test['close'].iloc[R]

        # if statement to assign 1 or 0 to new column depending on value of n
        if n >= 0:
            test['price_difference'].iloc[num] = 1
        else:
            test['price_difference'].iloc[num] = 0

        # Increase values by one for each iteration
        num = int(num) + 1
        T = int(T) + 1
        R = int(R) + 1
        
except:
    pass

IndexError: single positional indexer is out-of-bounds

In [15]:
test.head()

,time,avg_positive,avg_neutral,avg_negative,avg_compound,low,high,open,close,volume,price_difference
0,2018-12-31,0.138213,0.791725,0.070078,0.133069,3625.00,3835.40,3829.00,3691.86,12799.942626,
1,2019-01-01,0.133899,0.801962,0.064177,0.131864,3651.02,3841.17,3691.87,3826.10,10812.884988,1
2,2019-01-02,0.124542,0.805365,0.070119,0.111221,3770.07,3916.57,3826.10,3890.79,9982.470846,1
3,2019-01-03,0.145012,0.785654,0.069367,0.141588,3758.07,3893.80,3890.80,3787.57,9327.647089,0
4,2019-01-04,0.124144,0.801732,0.074139,0.123028,3730.00,3849.00,3787.57,3820.82,9225.150500,1


In [16]:
#test.to_csv('../Data/cleaned_sentiment_scores_same_day.csv')